<a href="https://colab.research.google.com/github/stigsfoot/google-ai-sprint/blob/main/docs/learner-resources/genui-adk-colab-tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Building Generative UI Systems with Google ADK

## Interactive Tutorial: From Tools to Multi-Agent Orchestration

Welcome to the hands-on tutorial for building generative UI systems using Google's Agent Development Kit (ADK). This notebook will guide you through creating UI components as ADK tools and building a multi-agent architecture that generates business intelligence dashboards.

### 🎯 Learning Objectives
By the end of this tutorial, you will:
- ✅ Understand UI components as callable ADK tools
- ✅ Create specialized agents with focused responsibilities  
- ✅ Implement tool composition for complex UI generation
- ✅ Master ADK delegation and evaluation patterns
- ✅ Debug and optimize multi-agent systems

### 🔗 Companion Resources
- **GenUI-ADK Learning Companion**: Your personalized AI tutor (Gemini Gem)
- **Full Documentation**: [ADK Integration Guide](../adk-integration-guide.md)
- **Code Examples**: [Implementation Examples](implementation-examples.py)


## 📋 Prerequisites Check

Before we begin, let's ensure you have everything needed:

In [ ]:
# Prerequisites Check
import sys
print(f"✅ Python Version: {sys.version}")
print("\n📋 Required for this tutorial:")
print("   🔑 Google AI Studio API Key (you'll be prompted to enter this securely)")
print("   🧠 Basic Python knowledge")
print("   ⚛️ Familiarity with React/JSX (helpful but not required)")
print("   🤖 Understanding of AI agents (we'll teach the ADK specifics)")

print("\n🎓 Learning Style Assessment:")
print("Think about how you learn best:")
print("   📊 Visual: You like diagrams and flowcharts")
print("   🛠️ Hands-on: You prefer coding immediately")
print("   📚 Theory-first: You want concepts before implementation")
print("   💡 Example-driven: You learn from concrete use cases")
print("\nThis tutorial adapts to all styles - let's begin!")

## 🔧 Step 1: Environment Setup & Installation

Let's start by installing the necessary libraries and setting up our environment.

In [ ]:
# Install Google ADK and dependencies with compatibility fixes
print("📦 Installing packages for Google Colab compatibility...")
print("⚠️  Note: You may see dependency warnings - these are expected in Colab")

# Install with specific version handling for Colab
!pip install --no-deps google-adk
!pip install python-dotenv -q

# Install core dependencies that ADK needs
!pip install google-generativeai>=0.3.0 -q
!pip install pydantic>=2.0.0 -q

print("\n📦 Installed packages:")
print("   ✅ google-adk: Google's Agent Development Kit")
print("   ✅ google-generativeai: Core AI functionality")
print("   ✅ python-dotenv: Environment variable management")
print("   ✅ pydantic: Data validation")

# Verify installation
try:
    import google.adk
    print("\n✅ ADK import successful!")
except ImportError as e:
    print(f"\n❌ ADK import failed: {e}")
    print("🔄 Trying alternative installation...")
    !pip install google-adk --force-reinstall -q

print("\n🎉 Installation complete!")
print("💡 If you see dependency warnings, they won't affect the tutorial functionality.")

In [ ]:
# Import necessary libraries
import os
import json
import asyncio
from getpass import getpass
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai.types import Content, Part

print("📚 Libraries imported successfully!")
print("\n🧠 Key ADK Components:")
print("   🤖 LlmAgent: The core agent class")
print("   🏃 Runner: Executes agent conversations")
print("   💾 InMemorySessionService: Manages conversation state")
print("   💬 Content & Part: Message formatting")

In [ ]:
# Secure API Key Setup
print("🔐 Secure API Key Configuration")
print("\nYou'll need a Google AI Studio API Key:")
print("1. Visit: https://makersuite.google.com/app/apikey")
print("2. Create a new API key")
print("3. Enter it securely below (it won't be displayed)")
print("\n⚠️  Important: Keep your API key private and secure!")

# Prompt for API key securely
api_key = getpass("Enter your Google AI Studio API Key: ")
os.environ["GOOGLE_API_KEY"] = api_key

# Verify API key is set (without displaying it)
if os.getenv('GOOGLE_API_KEY'):
    print("\n✅ API Key configured successfully!")
    print(f"   🔢 Length: {len(api_key)} characters")
    print(f"   🎭 Masked: {api_key[:8]}...{api_key[-4:]}")
else:
    print("\n❌ API Key not configured. Please run this cell again.")

## 🛠️ Step 2: Your First UI Tool

In GenUI-ADK, UI components are created as **tools** - regular Python functions that return JSX strings. Let's build your first tool!

### 🔑 Key Concepts:
- **No decorators**: ADK tools are plain Python functions (no `@Tool` decorator)
- **Return JSX strings**: Tools generate React component code as strings
- **Business context**: Tools should solve real business problems
- **Accessibility**: Always consider inclusive design

In [ ]:
# Your First UI Tool: Metric Card Generator
def create_metric_card_tool(value: str, label: str, change: str, context: str) -> str:
    """
    🎯 Generates a key metric card component with change indicator.
    
    This is your first ADK tool! Notice:
    - No @Tool decorator (ADK doesn't use them)
    - Clear docstring describing the tool's purpose
    - Type hints for better code quality
    - Returns a JSX string (React component code)
    
    Args:
        value: The main metric value to display (e.g., "$500,000")
        label: The title/label for the metric (e.g., "Total Revenue")
        change: Change from previous period (e.g., "+10%", "-5%")
        context: Additional context (e.g., "vs. last quarter")
    
    Returns:
        JSX string representing a styled metric card component
    """
    
    # 🎨 Dynamic styling based on change direction
    if change.startswith("+"):
        change_color = "green"
        trend_icon = "📈"
    elif change.startswith("-"):
        change_color = "red" 
        trend_icon = "📉"
    else:
        change_color = "gray"
        trend_icon = "➡️"
    
    # 🏗️ Generate JSX component string
    jsx_component = f'''
    <Card className="p-6 text-center hover:shadow-lg transition-shadow">
      <CardContent className="pt-6">
        <div className="text-4xl font-bold text-gray-900 mb-2">{value}</div>
        <p className="text-sm text-gray-600 mb-3">{label}</p>
        <div className="flex items-center justify-center space-x-2">
          <span className="text-lg">{trend_icon}</span>
          <Badge variant="{change_color}" className="font-semibold">
            {change}
          </Badge>
        </div>
        <p className="text-xs text-gray-500 mt-3">{context}</p>
        <div className="mt-4 w-full bg-gray-200 rounded-full h-2">
          <div className="bg-{change_color}-500 h-2 rounded-full" style="width: 75%"></div>
        </div>
      </CardContent>
    </Card>
    '''.strip()
    
    return jsx_component

# 🧪 Test your tool!
print("🧪 Testing your first UI tool...")
test_jsx = create_metric_card_tool(
    value="1,250",
    label="Active Users", 
    change="-5%",
    context="vs. last month"
)

print("\n✅ Tool executed successfully!")
print(f"📏 Generated JSX length: {len(test_jsx)} characters")
print(f"🔍 Contains Card component: {'<Card' in test_jsx}")
print(f"🎨 Dynamic styling applied: {'red' in test_jsx}")

print("\n📋 Generated JSX Preview:")
print("-" * 50)
print(test_jsx[:200] + "..." if len(test_jsx) > 200 else test_jsx)
print("-" * 50)

## 🤖 Step 3: Create Your First Agent

Now let's create an ADK agent that can use your tool. The agent will analyze user requests and decide when to call your tool.

In [ ]:
# Create your first specialized agent
def create_metrics_agent():
    """
    🎯 Creates a specialized agent for generating metric cards and KPI visualizations.
    
    This agent demonstrates:
    - Clear agent identity and purpose
    - Tool registration (no decorators needed)
    - Instruction prompt engineering
    - Specific model selection
    """
    
    # 📝 Agent instruction prompt (this is crucial!)
    agent_instruction = """
    You are a specialized Metrics Visualization Agent for business intelligence dashboards.
    
    YOUR ROLE:
    - Generate metric cards and KPI visualizations using available UI tools
    - Analyze business metrics and determine appropriate visual representation
    - Focus on clear, actionable insights for business users
    
    AVAILABLE TOOLS:
    - create_metric_card_tool: Generates metric cards with change indicators
    
    TOOL USAGE GUIDELINES:
    1. Extract key metrics from user queries
    2. Determine appropriate change indicators (+/- percentages)
    3. Provide meaningful context for business decisions
    4. Use consistent formatting for professional appearance
    
    When users ask about metrics, KPIs, or performance indicators, use your tools to generate appropriate visualizations.
    """.strip()
    
    # 🏗️ Create the LlmAgent
    metrics_agent = LlmAgent(
        name="metrics_visualization_agent",
        model="gemini-1.5-flash",  # Fast model for responsive UI generation
        instruction=agent_instruction,
        tools=[
            create_metric_card_tool  # Your tool function (not a string!)
        ]
    )
    
    return metrics_agent

# 🚀 Create your agent instance
print("🤖 Creating your specialized metrics agent...")
my_agent = create_metrics_agent()

print("\n✅ Agent created successfully!")
print(f"🏷️  Agent Name: {my_agent.name}")
print(f"🧠 Model: {my_agent.model}")
print(f"🛠️  Available Tools: {len(my_agent.tools)}")
print(f"📝 Instruction Length: {len(my_agent.instruction)} characters")

# 🔍 Inspect tool registration
if my_agent.tools:
    tool_names = [tool.__name__ for tool in my_agent.tools]
    print(f"🔧 Registered Tools: {tool_names}")
else:
    print("⚠️  No tools registered - check your agent configuration!")

## 🏃 Step 4: Set Up the ADK Runner

The Runner is the engine that executes your agent and manages conversations. Let's configure it properly.

In [ ]:
# Configure the ADK Runner
print("🏃 Setting up ADK Runner for agent execution...")

# 💾 Session management for conversation state
session_service = InMemorySessionService()
print("✅ Session service initialized")

# 🏗️ Create the runner with your agent
runner = Runner(
    agent=my_agent,
    session_service=session_service,
    app_name="genui_tutorial_demo"  # Identifies your application
)
print("✅ ADK Runner configured")

# 🧪 Runner health check
print("\n🔍 Runner Configuration:")
print(f"   🤖 Agent: {runner.agent.name}")
print(f"   📱 App Name: genui_tutorial_demo")
print(f"   💾 Session Service: {type(session_service).__name__}")
print(f"   🌐 API Key Configured: {'✅' if os.getenv('GOOGLE_API_KEY') else '❌'}")

print("\n🎉 ADK Runner is ready for agent execution!")

## 🎯 Step 5: Your First Agent Interaction

Now for the exciting part - let's interact with your agent and watch it generate UI components!

In [ ]:
# Define the agent interaction function
async def chat_with_agent(query: str, user_id: str = "tutorial_user", session_id: str = "session_001") -> str:
    """
    🎯 Sends a query to your agent and returns the generated UI component.
    
    This function demonstrates:
    - Proper ADK message formatting
    - Async execution patterns
    - Event streaming and response handling
    - Error handling for robust applications
    """
    
    print(f"\n🎯 User Query: '{query}'")
    print("🔄 Processing with your agent...")
    
    try:
        # 📨 Format message for ADK
        message = Content(
            role="user", 
            parts=[Part(text=query)]
        )
        
        # 🏃 Execute agent with async streaming
        final_response = "No response generated"
        
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session_id, 
            new_message=message
        ):
            # 🎯 Look for the final agent response
            if event.is_final_response() and event.content:
                final_response = event.content.parts[0].text
                break
        
        print("✅ Agent execution completed!")
        return final_response
        
    except Exception as e:
        error_msg = f"❌ Agent execution failed: {str(e)}"
        print(error_msg)
        return error_msg

print("🎮 Agent interaction function ready!")
print("\n🚀 Let's test your agent with a real business query...")

In [ ]:
# 🎯 Test your agent with a business query
business_query = "Show me a metric card for our current Monthly Recurring Revenue. The value is $45,000, which is a +12% increase from last month."

print("🚀 Executing your first agent interaction...")
print("🔄 This may take 5-10 seconds as the agent analyzes your query and generates UI code...")

# Execute the agent
generated_ui = await chat_with_agent(business_query)

# 📊 Analyze the results
print("\n" + "="*60)
print("🎉 AGENT RESPONSE ANALYSIS")
print("="*60)

if "<Card" in generated_ui:
    print("✅ SUCCESS: Agent generated a valid Card component!")
    print(f"📏 Response length: {len(generated_ui)} characters")
    print(f"🎨 Contains styling: {'className' in generated_ui}")
    print(f"📈 Includes metrics: {'45,000' in generated_ui}")
    print(f"📊 Shows change: {'+12%' in generated_ui}")
else:
    print("⚠️  Agent response doesn't contain expected UI component")
    print("🔍 This might be normal - the agent might be explaining its approach")

print("\n📋 FULL AGENT RESPONSE:")
print("-" * 60)
print(generated_ui)
print("-" * 60)

print("\n🎓 What just happened?")
print("   1. 🧠 Your agent analyzed the business query")
print("   2. 🎯 It identified the need for a metric card")
print("   3. 🛠️  It called your create_metric_card_tool")
print("   4. 🎨 The tool generated styled JSX code")
print("   5. 📤 The agent returned the UI component")

## 🔧 Step 6: Add More Tools (Progressive Complexity)

Let's expand your agent's capabilities by adding more sophisticated tools. This demonstrates tool composition and specialization.

In [ ]:
# 📊 Advanced Tool: Trend Line Chart Generator
def create_trend_chart_tool(data: list, title: str, trend_direction: str, insight: str) -> str:
    """
    🎯 Generates a trend line chart component with contextual insights.
    
    This tool demonstrates:
    - Complex data handling (list of data points)
    - Conditional styling based on trend direction
    - Integration of chart components
    - Business insight inclusion
    
    Args:
        data: List of data points [{'x': date, 'y': value}, ...]
        title: Chart title
        trend_direction: 'up', 'down', or 'stable'
        insight: Business insight about the trend
    """
    
    # 🎨 Dynamic styling based on trend
    color_map = {
        'up': {'color': 'green', 'icon': '📈', 'bg': 'green-50'},
        'down': {'color': 'red', 'icon': '📉', 'bg': 'red-50'},
        'stable': {'color': 'blue', 'icon': '➡️', 'bg': 'blue-50'}
    }
    
    theme = color_map.get(trend_direction, color_map['stable'])
    
    # 📊 Serialize data for JSX
    data_json = json.dumps(data)
    
    jsx_component = f'''
    <Card className="p-6 border-l-4 border-l-{theme['color']}-500">
      <CardHeader className="flex flex-row items-center space-y-0 pb-2">
        <span className="text-2xl mr-3">{theme['icon']}</span>
        <CardTitle className="text-lg font-semibold">{title}</CardTitle>
      </CardHeader>
      <CardContent>
        <div className="bg-{theme['bg']} rounded-lg p-4 mb-4">
          <LineChart 
            data={{{data_json}}} 
            className="h-48 w-full"
            strokeColor="{theme['color']}-500"
          />
        </div>
        <Alert className="border-{theme['color']}-200 bg-{theme['bg']}">
          <AlertDescription className="text-{theme['color']}-800 font-medium">
            💡 {insight}
          </AlertDescription>
        </Alert>
        <div className="mt-3 flex justify-between text-sm text-gray-600">
          <span>Data Points: {len(data)}</span>
          <span>Trend: {trend_direction.title()}</span>
        </div>
      </CardContent>
    </Card>
    '''.strip()
    
    return jsx_component

# 🧪 Test the new tool
print("🧪 Testing advanced trend chart tool...")

sample_data = [
    {'x': 'Jan', 'y': 100},
    {'x': 'Feb', 'y': 120},
    {'x': 'Mar', 'y': 140},
    {'x': 'Apr', 'y': 165}
]

trend_jsx = create_trend_chart_tool(
    data=sample_data,
    title="Monthly Revenue Growth",
    trend_direction="up",
    insight="Revenue has grown consistently over the past 4 months, indicating strong market demand."
)

print("✅ Advanced tool test successful!")
print(f"📊 Data points processed: {len(sample_data)}")
print(f"🎨 Dynamic styling applied: {'green' in trend_jsx}")
print(f"📈 Chart component included: {'LineChart' in trend_jsx}")

In [ ]:
# 🤖 Enhanced Agent with Multiple Tools
def create_enhanced_business_agent():
    """
    🎯 Creates an enhanced agent with multiple UI generation tools.
    
    This demonstrates:
    - Multi-tool agent configuration
    - Enhanced instruction prompts
    - Tool selection logic
    - Business intelligence focus
    """
    
    enhanced_instruction = """
    You are an Advanced Business Intelligence UI Generator.
    
    YOUR CAPABILITIES:
    - Generate metric cards for KPIs and performance indicators
    - Create trend line charts for temporal data analysis
    - Provide business insights and actionable recommendations
    
    AVAILABLE TOOLS:
    1. create_metric_card_tool: For individual metrics with change indicators
    2. create_trend_chart_tool: For temporal data and trend visualization
    
    TOOL SELECTION GUIDELINES:
    - Use metric cards for: single values, KPIs, performance indicators, comparisons
    - Use trend charts for: time series data, growth analysis, pattern identification
    - Always extract meaningful business insights
    - Focus on actionable information for decision makers
    
    When users ask for business dashboards or analytics, analyze their needs and select the most appropriate tool(s).
    """.strip()
    
    enhanced_agent = LlmAgent(
        name="enhanced_business_intelligence_agent",
        model="gemini-1.5-flash",
        instruction=enhanced_instruction,
        tools=[
            create_metric_card_tool,
            create_trend_chart_tool
        ]
    )
    
    return enhanced_agent

# 🚀 Create enhanced agent
print("🚀 Creating enhanced business intelligence agent...")
enhanced_agent = create_enhanced_business_agent()

# 🔄 Update runner with enhanced agent
runner_enhanced = Runner(
    agent=enhanced_agent,
    session_service=session_service,
    app_name="enhanced_genui_demo"
)

print("\n✅ Enhanced agent ready!")
print(f"🤖 Agent: {enhanced_agent.name}")
print(f"🛠️  Available Tools: {len(enhanced_agent.tools)}")
tool_names = [tool.__name__ for tool in enhanced_agent.tools]
print(f"🔧 Tool Registry: {tool_names}")

print("\n🎯 Your agent can now:")
print("   📊 Generate metric cards for KPIs")
print("   📈 Create trend charts for time series data")
print("   🧠 Select appropriate tools based on query analysis")
print("   💡 Provide business insights and recommendations")

## 🎯 Step 7: Advanced Agent Testing

Let's test your enhanced agent with complex business scenarios to see tool selection in action.

In [ ]:
# 🎯 Advanced Agent Testing Function
async def test_enhanced_agent(query: str, test_name: str) -> dict:
    """
    🧪 Tests the enhanced agent and analyzes the response.
    
    Returns detailed analysis of agent behavior for learning purposes.
    """
    
    print(f"\n🧪 TEST: {test_name}")
    print("=" * 50)
    print(f"📝 Query: {query}")
    print("🔄 Processing...")
    
    # Create new session for each test
    import uuid
    session_id = f"test_{uuid.uuid4().hex[:8]}"
    
    try:
        # Execute agent
        message = Content(role="user", parts=[Part(text=query)])
        
        response = ""
        async for event in runner_enhanced.run_async(
            user_id="test_user",
            session_id=session_id,
            new_message=message
        ):
            if event.is_final_response() and event.content:
                response = event.content.parts[0].text
                break
        
        # Analyze response
        analysis = {
            'test_name': test_name,
            'query': query,
            'response': response,
            'response_length': len(response),
            'contains_jsx': '<Card' in response or '<div' in response,
            'metric_card_used': 'create_metric_card_tool' in response or ('Badge' in response and 'CardContent' in response),
            'trend_chart_used': 'LineChart' in response or 'trend' in response.lower(),
            'business_insight': 'insight' in response.lower() or 'recommend' in response.lower(),
            'success': True
        }
        
        # Print analysis
        print("\n📊 RESPONSE ANALYSIS:")
        print(f"   ✅ Response Generated: {analysis['success']}")
        print(f"   📏 Length: {analysis['response_length']} characters")
        print(f"   🎨 Contains JSX: {analysis['contains_jsx']}")
        print(f"   📊 Metric Card Tool: {analysis['metric_card_used']}")
        print(f"   📈 Trend Chart Tool: {analysis['trend_chart_used']}")
        print(f"   💡 Business Insights: {analysis['business_insight']}")
        
        return analysis
        
    except Exception as e:
        print(f"❌ Test failed: {str(e)}")
        return {
            'test_name': test_name,
            'query': query, 
            'error': str(e),
            'success': False
        }

print("🧪 Advanced testing framework ready!")
print("Let's run comprehensive tests on your enhanced agent...")

In [ ]:
# 🎯 Test Suite: Multiple Business Scenarios
test_scenarios = [
    {
        'name': 'KPI Metric Request',
        'query': 'Show me our customer satisfaction score. Current value is 4.8/5.0, which is +0.3 improvement from last quarter.'
    },
    {
        'name': 'Trend Analysis Request', 
        'query': 'Create a trend chart showing our monthly user growth over the past 6 months. The data shows steady upward growth.'
    },
    {
        'name': 'Complex Business Dashboard',
        'query': 'I need a dashboard component for our quarterly revenue. We made $2.1M this quarter, up 15% from last quarter. Show the growth trend.'
    }
]

print("🚀 Starting comprehensive agent testing...")
print(f"📋 Running {len(test_scenarios)} test scenarios...")

test_results = []

for i, scenario in enumerate(test_scenarios, 1):
    print(f"\n🔄 Running Test {i}/{len(test_scenarios)}...")
    
    result = await test_enhanced_agent(
        query=scenario['query'],
        test_name=scenario['name']
    )
    
    test_results.append(result)
    
    # Show response preview
    if result.get('success') and result.get('response'):
        response_preview = result['response'][:200] + "..." if len(result['response']) > 200 else result['response']
        print(f"\n📋 Response Preview:")
        print("-" * 40)
        print(response_preview)
        print("-" * 40)

# 📊 Test Suite Summary
print("\n" + "=" * 60)
print("🎉 TEST SUITE COMPLETE - SUMMARY REPORT")
print("=" * 60)

successful_tests = [r for r in test_results if r.get('success', False)]
failed_tests = [r for r in test_results if not r.get('success', False)]

print(f"✅ Successful Tests: {len(successful_tests)}/{len(test_results)}")
print(f"❌ Failed Tests: {len(failed_tests)}")

if successful_tests:
    jsx_responses = [r for r in successful_tests if r.get('contains_jsx', False)]
    metric_card_usage = [r for r in successful_tests if r.get('metric_card_used', False)]
    trend_chart_usage = [r for r in successful_tests if r.get('trend_chart_used', False)]
    business_insights = [r for r in successful_tests if r.get('business_insight', False)]
    
    print(f"\n📊 Tool Usage Analysis:")
    print(f"   🎨 JSX Generation: {len(jsx_responses)}/{len(successful_tests)}")
    print(f"   📊 Metric Card Tool: {len(metric_card_usage)}/{len(successful_tests)}")
    print(f"   📈 Trend Chart Tool: {len(trend_chart_usage)}/{len(successful_tests)}")
    print(f"   💡 Business Insights: {len(business_insights)}/{len(successful_tests)}")

print("\n🎓 What did we learn?")
print("   1. 🧠 Your agent can analyze different query types")
print("   2. 🛠️  It selects appropriate tools based on context")
print("   3. 🎨 It generates valid UI components consistently")
print("   4. 💡 It provides business value through insights")
print("\n🚀 You've successfully built a multi-tool generative UI agent!")

## 🏗️ Step 8: Multi-Agent Architecture (Advanced)

Now let's build a true multi-agent system with specialization and delegation - the core pattern from the full project!

In [ ]:
# 🏗️ Multi-Agent Architecture: Specialized Agents

def create_accessibility_agent():
    """
    ♿ Creates a specialized agent for accessibility-focused UI components.
    
    This demonstrates specialization in multi-agent systems.
    """
    
    def create_accessible_metric_tool(value: str, label: str, change: str) -> str:
        """Generate WCAG-compliant metric cards with high contrast and ARIA labels."""
        
        change_color = "green" if change.startswith("+") else "red"
        
        return f'''
        <Card className="border-4 border-black bg-yellow-50 high-contrast">
          <CardHeader className="bg-black text-white p-4">
            <CardTitle className="text-xl font-bold" aria-label="{label} metric">
              ♿ {label} (Accessible)
            </CardTitle>
          </CardHeader>
          <CardContent className="p-6 text-center">
            <div className="text-5xl font-bold text-black mb-4" aria-label="Current value {value}">
              {value}
            </div>
            <Badge className="bg-{change_color}-800 text-white text-lg p-3" aria-label="Change {change}">
              {change}
            </Badge>
            <div className="mt-4 p-3 bg-black text-white text-sm rounded">
              <strong>Accessibility Features:</strong> High contrast, ARIA labels, keyboard navigation
            </div>
          </CardContent>
        </Card>
        '''.strip()
    
    return LlmAgent(
        name="accessibility_agent",
        model="gemini-1.5-flash",
        instruction="""
        You are an Accessibility Specialist Agent focused on creating WCAG-compliant UI components.
        
        PRIORITIES:
        - High contrast color schemes
        - Proper ARIA labels and semantic markup  
        - Screen reader compatibility
        - Keyboard navigation support
        
        Always prioritize accessibility and inclusive design.
        """,
        tools=[create_accessible_metric_tool]
    )

def create_root_orchestrator():
    """
    🎯 Creates the root orchestrator agent that delegates to specialized agents.
    
    This is the key pattern for multi-agent systems in ADK.
    """
    
    return LlmAgent(
        name="generative_ui_orchestrator",
        model="gemini-1.5-flash",
        instruction="""
        You are the Root Orchestrator for a multi-agent generative UI system.
        
        YOUR ROLE:
        - Analyze business queries for visualization requirements
        - Delegate to appropriate specialized sub-agents
        - Coordinate responses from multiple agents
        
        AVAILABLE SUB-AGENTS:
        - enhanced_business_intelligence_agent: Standard business metrics and charts
        - accessibility_agent: WCAG-compliant accessible components
        
        DELEGATION STRATEGY:
        - Use business intelligence agent for standard metrics and charts
        - Use accessibility agent when accessibility is explicitly requested
        - Always prioritize user experience and business value
        
        Analyze each query and delegate to the most appropriate specialist.
        """,
        sub_agents=[
            enhanced_agent,
            create_accessibility_agent()
        ]
    )

# 🚀 Create multi-agent system
print("🏗️ Building multi-agent architecture...")

root_orchestrator = create_root_orchestrator()
multi_agent_runner = Runner(
    agent=root_orchestrator,
    session_service=session_service,
    app_name="multi_agent_genui_demo"
)

print("\n✅ Multi-agent system ready!")
print(f"🎯 Root Agent: {root_orchestrator.name}")
print(f"🤖 Sub-agents: {len(root_orchestrator.sub_agents)}")

if hasattr(root_orchestrator, 'sub_agents') and root_orchestrator.sub_agents:
    sub_agent_names = [agent.name for agent in root_orchestrator.sub_agents]
    print(f"🔧 Specialist Agents: {sub_agent_names}")
    
    total_tools = sum(len(agent.tools) if hasattr(agent, 'tools') and agent.tools else 0 
                     for agent in root_orchestrator.sub_agents)
    print(f"🛠️  Total Tools Available: {total_tools}")

print("\n🎯 Your multi-agent system can now:")
print("   🧠 Analyze queries for specialization needs")
print("   🎯 Delegate to appropriate specialist agents")
print("   📊 Generate standard business visualizations")
print("   ♿ Create accessibility-compliant components")
print("   🔄 Coordinate complex multi-agent workflows")

In [ ]:
# 🎯 Test Multi-Agent Delegation
async def test_multi_agent_delegation():
    """
    🧪 Tests the multi-agent system with different query types to observe delegation.
    """
    
    delegation_tests = [
        {
            'name': 'Standard Business Query',
            'query': 'Show me our quarterly revenue performance: $500K, up 8% from last quarter',
            'expected_agent': 'business_intelligence_agent'
        },
        {
            'name': 'Accessibility-Focused Query',
            'query': 'Create an accessible metric card for our user engagement rate with high contrast colors for visually impaired users',
            'expected_agent': 'accessibility_agent'
        },
        {
            'name': 'Complex Multi-Agent Query',
            'query': 'Generate a dashboard showing our customer satisfaction (4.2/5, +0.5 improvement) with accessibility features for screen readers',
            'expected_agent': 'both_agents'
        }
    ]
    
    print("🚀 Testing Multi-Agent Delegation Patterns...")
    print(f"📋 Running {len(delegation_tests)} delegation tests...")
    
    for i, test in enumerate(delegation_tests, 1):
        print(f"\n🧪 Delegation Test {i}: {test['name']}")
        print("=" * 50)
        print(f"📝 Query: {test['query']}")
        print(f"🎯 Expected: {test['expected_agent']}")
        print("🔄 Processing with multi-agent system...")
        
        try:
            message = Content(role="user", parts=[Part(text=test['query'])])
            
            response = ""
            async for event in multi_agent_runner.run_async(
                user_id="delegation_test",
                session_id=f"delegation_test_{i}",
                new_message=message
            ):
                if event.is_final_response() and event.content:
                    response = event.content.parts[0].text
                    break
            
            # Analyze delegation behavior
            print("\n📊 DELEGATION ANALYSIS:")
            
            # Check for accessibility features
            accessibility_indicators = [
                'aria-label' in response.lower(),
                'high-contrast' in response.lower(),
                'accessible' in response.lower(),
                'wcag' in response.lower(),
                'screen reader' in response.lower()
            ]
            
            # Check for business intelligence features
            business_indicators = [
                'metric' in response.lower(),
                'badge' in response.lower(),
                'chart' in response.lower(),
                'trend' in response.lower()
            ]
            
            accessibility_score = sum(accessibility_indicators)
            business_score = sum(business_indicators)
            
            print(f"   ♿ Accessibility Features: {accessibility_score}/5")
            print(f"   📊 Business Intelligence Features: {business_score}/4")
            print(f"   📏 Response Length: {len(response)} characters")
            print(f"   🎨 Contains JSX: {'<Card' in response}")
            
            # Determine likely delegation
            if accessibility_score > business_score:
                likely_agent = "accessibility_agent"
            elif business_score > accessibility_score:
                likely_agent = "business_intelligence_agent"
            else:
                likely_agent = "orchestrator_or_both"
            
            print(f"   🎯 Likely Delegation: {likely_agent}")
            
            # Show response preview
            preview = response[:150] + "..." if len(response) > 150 else response
            print(f"\n📋 Response Preview:")
            print("-" * 30)
            print(preview)
            print("-" * 30)
            
        except Exception as e:
            print(f"❌ Delegation test failed: {str(e)}")
    
    print("\n" + "=" * 60)
    print("🎉 MULTI-AGENT DELEGATION TESTING COMPLETE")
    print("=" * 60)
    print("\n🎓 Key Learnings:")
    print("   1. 🧠 Root orchestrator analyzes query context")
    print("   2. 🎯 Delegation happens based on query requirements")
    print("   3. 🤖 Specialized agents provide domain expertise")
    print("   4. 🔄 Complex queries may involve multiple agents")
    print("   5. 📊 Response quality improves with specialization")

# Run delegation tests
await test_multi_agent_delegation()

## 🔍 Step 9: Debugging & Optimization

Learn essential debugging techniques for ADK agents and UI generation systems.

In [ ]:
# 🔍 Debugging Toolkit for ADK Agents

def debug_agent_configuration(agent):
    """
    🔧 Analyzes agent configuration for common issues.
    """
    print(f"🔍 DEBUGGING AGENT: {agent.name}")
    print("=" * 40)
    
    # Check basic configuration
    print(f"✅ Agent Name: {agent.name}")
    print(f"🧠 Model: {agent.model}")
    print(f"📝 Instruction Length: {len(agent.instruction)} chars")
    
    # Check tools
    if hasattr(agent, 'tools') and agent.tools:
        print(f"🛠️  Tool Count: {len(agent.tools)}")
        for i, tool in enumerate(agent.tools):
            print(f"   {i+1}. {tool.__name__}")
            # Check tool function signature
            import inspect
            sig = inspect.signature(tool)
            print(f"      Parameters: {list(sig.parameters.keys())}")
    else:
        print("⚠️  No tools registered")
    
    # Check sub-agents
    if hasattr(agent, 'sub_agents') and agent.sub_agents:
        print(f"🤖 Sub-agents: {len(agent.sub_agents)}")
        for i, sub_agent in enumerate(agent.sub_agents):
            print(f"   {i+1}. {sub_agent.name}")
    else:
        print("📋 No sub-agents (leaf agent)")
    
    print()

def debug_tool_output(tool_function, *args, **kwargs):
    """
    🧪 Tests a tool function in isolation.
    """
    print(f"🧪 TESTING TOOL: {tool_function.__name__}")
    print("=" * 40)
    
    try:
        result = tool_function(*args, **kwargs)
        
        print("✅ Tool execution successful")
        print(f"📏 Output length: {len(result)} characters")
        print(f"🎨 Contains JSX: {'<' in result and '>' in result}")
        print(f"🏷️  Contains className: {'className' in result}")
        
        # JSX validation
        jsx_tags = ['<Card', '<div', '<span', '<Badge']
        found_tags = [tag for tag in jsx_tags if tag in result]
        print(f"🔧 JSX Tags Found: {found_tags}")
        
        # Show preview
        preview = result[:200] + "..." if len(result) > 200 else result
        print(f"\n📋 Output Preview:")
        print("-" * 30)
        print(preview)
        print("-" * 30)
        
        return result
        
    except Exception as e:
        print(f"❌ Tool execution failed: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

def validate_jsx_syntax(jsx_string):
    """
    ✅ Basic JSX syntax validation.
    """
    print("✅ JSX SYNTAX VALIDATION")
    print("=" * 30)
    
    issues = []
    
    # Check for basic tag structure
    if jsx_string.count('<') != jsx_string.count('>'):
        issues.append("Mismatched angle brackets")
    
    # Check for unescaped quotes
    if '"""' in jsx_string or "'''" in jsx_string:
        issues.append("Potential unescaped quotes in attributes")
    
    # Check for className syntax
    if 'class=' in jsx_string and 'className=' not in jsx_string:
        issues.append("Use className instead of class in JSX")
    
    # Check for self-closing tags
    problem_tags = ['<img', '<input', '<br']
    for tag in problem_tags:
        if tag in jsx_string and not '/>' in jsx_string[jsx_string.find(tag):jsx_string.find(tag)+50]:
            issues.append(f"Potential self-closing tag issue: {tag}")
    
    if issues:
        print("⚠️  Issues found:")
        for issue in issues:
            print(f"   - {issue}")
    else:
        print("✅ No obvious syntax issues detected")
    
    return len(issues) == 0

print("🔍 Debugging toolkit ready!")
print("\nLet's debug your agents and tools...")

In [ ]:
# 🧪 Run Comprehensive Debugging Session
print("🚀 COMPREHENSIVE DEBUGGING SESSION")
print("=" * 50)

# Debug agent configurations
print("\n1. 🔍 AGENT CONFIGURATION ANALYSIS")
print("-" * 40)

debug_agent_configuration(enhanced_agent)
if hasattr(root_orchestrator, 'sub_agents') and root_orchestrator.sub_agents:
    debug_agent_configuration(root_orchestrator.sub_agents[1])  # Accessibility agent

# Debug tool functions
print("\n2. 🧪 TOOL FUNCTION TESTING")
print("-" * 40)

# Test metric card tool
metric_result = debug_tool_output(
    create_metric_card_tool,
    value="$1.2M",
    label="Monthly Revenue",
    change="+8%",
    context="vs. last month"
)

print("\n" + "-" * 40)

# Test trend chart tool
trend_result = debug_tool_output(
    create_trend_chart_tool,
    data=[{'x': 'Q1', 'y': 100}, {'x': 'Q2', 'y': 120}],
    title="Quarterly Growth",
    trend_direction="up",
    insight="Strong upward trend continues"
)

# JSX validation
print("\n3. ✅ JSX SYNTAX VALIDATION")
print("-" * 40)

if metric_result:
    print("Validating Metric Card JSX:")
    validate_jsx_syntax(metric_result)

if trend_result:
    print("\nValidating Trend Chart JSX:")
    validate_jsx_syntax(trend_result)

# Performance analysis
print("\n4. ⚡ PERFORMANCE ANALYSIS")
print("-" * 40)

import time

# Measure tool execution time
start_time = time.time()
for i in range(100):
    create_metric_card_tool("$100K", "Test Metric", "+5%", "test context")
end_time = time.time()

avg_time = (end_time - start_time) / 100 * 1000  # Convert to milliseconds
print(f"⚡ Average tool execution time: {avg_time:.2f}ms")

# Memory usage estimation
sample_jsx = create_metric_card_tool("$500K", "Revenue", "+10%", "quarterly")
jsx_size = len(sample_jsx.encode('utf-8'))
print(f"💾 Average JSX output size: {jsx_size} bytes")

print("\n" + "=" * 50)
print("🎉 DEBUGGING SESSION COMPLETE")
print("=" * 50)

print("\n🎓 Debugging Best Practices:")
print("   1. 🔍 Always validate agent configuration first")
print("   2. 🧪 Test tools in isolation before integration")
print("   3. ✅ Validate JSX syntax and structure")
print("   4. ⚡ Monitor performance for production use")
print("   5. 📊 Use ADK web interface for trace analysis")

print("\n🚀 Your debugging skills are now production-ready!")

## 🎓 Step 10: Learning Assessment & Next Steps

Let's assess what you've learned and explore advanced topics for continued growth.

In [ ]:
# 🎓 Learning Assessment & Certification

def run_learning_assessment():
    """
    🎯 Comprehensive assessment of tutorial concepts.
    """
    
    print("🎓 GENUI-ADK LEARNING ASSESSMENT")
    print("=" * 50)
    
    assessment_checklist = [
        {
            'category': '🛠️ Tool Development',
            'skills': [
                'Create UI tools as plain Python functions (no decorators)',
                'Generate valid JSX strings from tools',
                'Implement dynamic styling based on business logic',
                'Handle complex data structures in tools',
                'Include accessibility features in generated components'
            ]
        },
        {
            'category': '🤖 Agent Architecture', 
            'skills': [
                'Configure LlmAgent with proper instructions',
                'Register tools with agents correctly',
                'Write effective agent instruction prompts',
                'Understand agent specialization patterns',
                'Design multi-agent hierarchies with delegation'
            ]
        },
        {
            'category': '🏃 ADK Integration',
            'skills': [
                'Set up ADK Runner for agent execution',
                'Handle async execution patterns',
                'Format messages for ADK agents',
                'Process agent responses correctly',
                'Manage session state appropriately'
            ]
        },
        {
            'category': '🔍 Debugging & Optimization',
            'skills': [
                'Debug agent configuration issues',
                'Test tools in isolation',
                'Validate JSX syntax and structure',
                'Analyze agent performance',
                'Use debugging tools effectively'
            ]
        },
        {
            'category': '💡 Advanced Concepts',
            'skills': [
                'Design specialized agent domains',
                'Implement intelligent delegation patterns',
                'Create accessible UI components',
                'Handle complex business requirements',
                'Optimize for production deployment'
            ]
        }
    ]
    
    total_skills = sum(len(cat['skills']) for cat in assessment_checklist)
    
    print(f"📊 Assessment covers {total_skills} core skills across {len(assessment_checklist)} categories:\n")
    
    for category in assessment_checklist:
        print(f"{category['category']} ({len(category['skills'])} skills)")
        for skill in category['skills']:
            print(f"   ✅ {skill}")
        print()
    
    return assessment_checklist

def generate_completion_certificate():
    """
    🏆 Generates a completion certificate for the tutorial.
    """
    
    certificate = f"""
    ╔══════════════════════════════════════════════════════════════╗
    ║                    🏆 CERTIFICATE OF COMPLETION 🏆             ║
    ╠══════════════════════════════════════════════════════════════╣
    ║                                                              ║
    ║         GENERATIVE UI SYSTEMS WITH GOOGLE ADK                ║
    ║              Multi-Agent Architecture Tutorial                ║
    ║                                                              ║
    ║  This certifies that the participant has successfully        ║
    ║  completed the comprehensive tutorial covering:              ║
    ║                                                              ║
    ║  🛠️  UI Tool Development with ADK                            ║
    ║  🤖 Multi-Agent System Architecture                          ║
    ║  🎨 Dynamic JSX Component Generation                         ║
    ║  ♿ Accessibility-First Design Patterns                      ║
    ║  🔍 Advanced Debugging & Optimization                        ║
    ║                                                              ║
    ║  Skills Mastered: 25+ core competencies                     ║
    ║  Completion Date: {time.strftime('%B %d, %Y')}                      ║
    ║                                                              ║
    ║  🎓 Ready for Advanced GenUI-ADK Development                 ║
    ║                                                              ║
    ╚══════════════════════════════════════════════════════════════╝
    """
    
    return certificate

# Run assessment
assessment = run_learning_assessment()

print("\n🎉 CONGRATULATIONS!")
print("=" * 50)
print("You have successfully completed the GenUI-ADK Tutorial!")
print("\nYour journey included:")
print("   🚀 Built your first UI generation tool")
print("   🤖 Created specialized ADK agents")
print("   🏗️  Implemented multi-agent architecture")
print("   🔍 Mastered debugging techniques")
print("   ♿ Applied accessibility best practices")

# Generate certificate
certificate = generate_completion_certificate()
print(certificate)

import time

In [ ]:
# 🚀 Next Steps & Advanced Learning Paths

def show_next_steps():
    """
    🎯 Provides personalized next steps based on learning style and interests.
    """
    
    print("🚀 YOUR NEXT STEPS IN GENUI-ADK MASTERY")
    print("=" * 50)
    
    learning_paths = {
        '🏗️ Architecture Track': {
            'description': 'Focus on building complex multi-agent systems',
            'next_steps': [
                'Design custom specialized agents for new domains',
                'Implement advanced delegation patterns',
                'Build real-time data integration',
                'Create agent evaluation frameworks',
                'Deploy production ADK systems'
            ]
        },
        '🎨 UI/UX Track': {
            'description': 'Advance your component generation and design systems',
            'next_steps': [
                'Create custom design system integrations',
                'Build interactive component generators',
                'Implement responsive design patterns',
                'Master advanced accessibility features',
                'Develop mobile-first UI generation'
            ]
        },
        '📊 Data & Analytics Track': {
            'description': 'Specialize in business intelligence and data visualization',
            'next_steps': [
                'Integrate with real business data sources',
                'Build advanced chart and visualization tools',
                'Create predictive analytics components',
                'Implement real-time dashboard updates',
                'Design executive-level reporting systems'
            ]
        },
        '🔬 Research & Innovation Track': {
            'description': 'Push the boundaries of generative UI systems',
            'next_steps': [
                'Experiment with new ADK capabilities',
                'Contribute to open source ADK projects',
                'Research novel UI generation patterns',
                'Develop evaluation methodologies',
                'Publish findings and case studies'
            ]
        }
    }
    
    for track_name, track_info in learning_paths.items():
        print(f"\n{track_name}")
        print(f"📝 {track_info['description']}")
        print("Next Steps:")
        for i, step in enumerate(track_info['next_steps'], 1):
            print(f"   {i}. {step}")
    
    print("\n🎓 COMPANION RESOURCES")
    print("-" * 30)
    print("📚 Full Documentation: adk-integration-guide.md")
    print("🤖 GenUI-ADK Learning Companion: Your personalized AI tutor")
    print("💻 Code Examples: implementation-examples.py")
    print("🧪 Practice Exercises: learning-exercises.md")
    print("🔧 Troubleshooting: troubleshooting-guide.md")
    
    print("\n🌐 COMMUNITY & SUPPORT")
    print("-" * 30)
    print("💬 Join the GenUI-ADK community")
    print("🎯 Contribute to example projects")
    print("📖 Share your learning journey")
    print("🤝 Mentor other learners")
    
    return learning_paths

def create_personalized_action_plan():
    """
    🎯 Creates a personalized 30-day action plan.
    """
    
    action_plan = """
    🗓️  YOUR 30-DAY GENUI-ADK MASTERY PLAN
    ═══════════════════════════════════════════
    
    Week 1: Foundation Reinforcement
    ▸ Day 1-2: Practice building 3 custom UI tools
    ▸ Day 3-4: Create a specialized agent for your domain
    ▸ Day 5-7: Implement accessibility features in all components
    
    Week 2: Multi-Agent Systems
    ▸ Day 8-10: Design a 3-agent specialization hierarchy
    ▸ Day 11-12: Implement intelligent delegation patterns
    ▸ Day 13-14: Build evaluation framework for your agents
    
    Week 3: Real-World Integration
    ▸ Day 15-17: Connect to actual business data sources
    ▸ Day 18-19: Build a complete dashboard generation system
    ▸ Day 20-21: Implement user feedback and iteration cycles
    
    Week 4: Advanced Topics & Deployment
    ▸ Day 22-24: Optimize for production performance
    ▸ Day 25-26: Implement advanced debugging and monitoring
    ▸ Day 27-28: Deploy your system for real users
    ▸ Day 29-30: Document and share your learnings
    
    🎯 Success Metrics:
    ▸ 10+ custom UI tools created
    ▸ 5+ specialized agents implemented
    ▸ 1 complete multi-agent system deployed
    ▸ 3+ accessibility compliance validations
    ▸ 1 community contribution or case study
    """
    
    return action_plan

# Show next steps
learning_paths = show_next_steps()

# Create action plan
action_plan = create_personalized_action_plan()
print(action_plan)

print("\n🎉 FINAL MESSAGE")
print("=" * 50)
print("🚀 You are now equipped with the foundational skills to build")
print("   production-quality generative UI systems using Google ADK!")
print("\n🎓 Remember: The best way to master these concepts is through")
print("   hands-on practice with real business problems.")
print("\n💡 Your GenUI-ADK Learning Companion is always available to")
print("   provide personalized guidance as you continue your journey.")
print("\n🌟 Happy building, and welcome to the future of AI-powered UI generation!")

## 📚 Summary & Resources

### 🎯 What You've Accomplished

Congratulations! You've successfully completed a comprehensive tutorial on building generative UI systems with Google ADK. You now have hands-on experience with:

- **🛠️ UI Tool Development**: Creating React components as ADK tools
- **🤖 Agent Architecture**: Building specialized agents with clear responsibilities
- **🏗️ Multi-Agent Systems**: Implementing orchestration and delegation patterns
- **♿ Accessibility**: Creating inclusive, WCAG-compliant components
- **🔍 Debugging & Optimization**: Professional debugging and performance analysis

### 🔗 Related Resources

- **[Full Documentation](../adk-integration-guide.md)**: Complete walkthrough with evaluation frameworks
- **[Code Examples](implementation-examples.py)**: Production-ready agent implementations  
- **[Practice Exercises](learning-exercises.md)**: Progressive challenges for skill building
- **[Troubleshooting Guide](troubleshooting-guide.md)**: Solutions for common issues
- **GenUI-ADK Learning Companion**: Your personalized AI tutor (Gemini Gem)

### 🚀 Continue Your Journey

The skills you've learned here are the foundation for building sophisticated, AI-powered user interfaces. Whether you choose to focus on architecture, design, data visualization, or research, you're now equipped to create systems that bridge the gap between AI capabilities and human-centered design.

Remember: The future of UI development is generative, intelligent, and accessible by default. You're now part of building that future!

---

*This tutorial was created as part of the Google AI Sprint project demonstrating real-world ADK applications. For questions or contributions, engage with the GenUI-ADK Learning Companion or visit the project repository.*